In [1]:
import torch
import os
from ultralytics import YOLO
import cv2
import numpy as np

In [2]:
model = YOLO("../models/yolov8x.pt")

In [4]:
# Define input directory, output directory, and list of image files
input_dir = "../dataset/box_jpg"
output_dir = "../yolo_predictions/yolov8x/box"
image_files = os.listdir(input_dir)

# Check if the output directory exists, if not, create it
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

mask_color = (250, 200, 0)

# Iterate over each image file
for image_file in image_files:
    # Load the image
    img_path = os.path.join(input_dir, image_file)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    filename = os.path.basename(img_path) 

    # Perform inference on the image
    results = model.predict(source=img_path, conf=0.5, save=False, save_txt=False)
    masks = results[0].masks
    boxes = results[0].boxes

    # Apply the masks and boxes to the image
    for mask in masks:
        if mask is not None:
            m = torch.squeeze(mask.data)

            # Convert the tensor to numpy array
            mask_np = m.cpu().numpy().astype(np.uint8)

            # Create a colored mask
            colored_mask = np.zeros((mask_np.shape[0], mask_np.shape[1], 3), dtype=np.uint8)
            colored_mask[mask_np == 1] = mask_color

            # Resize mask to match input image size
            colored_mask = cv2.resize(colored_mask, (img.shape[1], img.shape[0]))

            # Blend the colored mask with the input image
            alpha = 0.5  # Transparency factor. You can change it as needed.
            overlayed_img = cv2.addWeighted(img, 1, colored_mask, alpha, 0)

            new_name = filename.replace('.jpg', '') + '_overlay' + '.jpg'

            # Save the image with overlayed mask
            cv2.imwrite(os.path.join(output_dir, new_name), cv2.cvtColor(overlayed_img, cv2.COLOR_RGB2BGR))


image 1/1 c:\Users\kimbe\Documents\GitHub\mandible-height-estimation\dataset\box_jpg\1506343.jpg: 320x640 1 mandible, 670.5ms
Speed: 2.5ms preprocess, 670.5ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

image 1/1 c:\Users\kimbe\Documents\GitHub\mandible-height-estimation\dataset\box_jpg\1506536.jpg: 320x640 1 mandible, 548.6ms
Speed: 1.3ms preprocess, 548.6ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

image 1/1 c:\Users\kimbe\Documents\GitHub\mandible-height-estimation\dataset\box_jpg\1507529.jpg: 320x640 1 mandible, 600.6ms
Speed: 2.0ms preprocess, 600.6ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

image 1/1 c:\Users\kimbe\Documents\GitHub\mandible-height-estimation\dataset\box_jpg\1508098.jpg: 320x640 1 mandible, 615.3ms
Speed: 1.0ms preprocess, 615.3ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

image 1/1 c:\Users\kimbe\Documents\GitHub\mandible-height-estimation\dataset\box_jpg\1508936.jpg: 3

TypeError: 'NoneType' object is not iterable